# TensorFlow Autoencodeur avec attention pour le PAr

## Setup

Moi j'ai installé tf addons par `pip install tensorflow-addons==0.13.0` (ET NON PAS `conda install -c esri tensorflow-addons`). Voir les compatibilités [sur le github de tensorflow_addons](https://github.com/tensorflow/addons).

In [1]:
# !pip install tensorflow-addons==0.11.2

In [2]:
import sys
print(sys.version)

3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:36:06) [MSC v.1929 64 bit (AMD64)]


In [3]:
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


import numpy as np
import os
import io
import time

print(tf.__version__)

2.7.0


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  1


# Step 1: Get the data

In [5]:
path_reglement_scol  = './word2vec_docs_scol_traités/corpus.txt'
path_questions_scol  = './word2vec_docs_scol_traités/toutes-les-questions.txt'

# Step 2: Preprocess the data

In [6]:
import re as regex
# acquisition du texte
reglement_scol = io.open(path_reglement_scol, encoding='UTF-8').read()#.strip().split('\n')
questions_scol = io.open(path_questions_scol, encoding='UTF-8').read()#.strip().split('\n')
texte = reglement_scol + ' ' + questions_scol
texte = regex.sub("\n", " ", texte)

On crée d'abord une liste de phrases dont chaque mot est séparé par un espace. On a besoin de `spacy` pour découper correctement les mots en français d'abord.

In [7]:
import nltk
import spacy
nlp = spacy.load('fr_core_news_sm')
phrases = nltk.tokenize.sent_tokenize(texte, language='french')
print('phrases parsées par NLTK')
phrasesTokeniseesSpacy = [nlp(s) for s in phrases]
print('phrases tokénisées par spacy')
phrasesSpacy = [' '.join(['<start> ']+[token.text.lower() for token in doc]+['<stop>']) for doc in phrasesTokeniseesSpacy]
print('phrases découpées en tokens puis refusionnées')

phrases parsées par NLTK
phrases tokénisées par spacy
phrases découpées en tokens puis refusionnées


On supprime les listes inutiles désormais

In [8]:
del phrasesTokeniseesSpacy
del phrases

Créer un tokéniseur adapté à notre vocabulaire

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters='')
# créer un tokenizer adapté à tout le vocabulaire des phrases
tokenizer.fit_on_texts(phrasesSpacy)

Créer les tenseurs pour toutes les phrases et padder le tout

In [10]:
tensor_sentences = tokenizer.texts_to_sequences(phrasesSpacy)
print(type(tensor_sentences))
print(phrasesSpacy[0],tensor_sentences[0])
# enfin on padd le tout pour pouvoir l'utiliser dans un réseau de neurones
tensor_sentences = tf.keras.preprocessing.sequence.pad_sequences(tensor_sentences,padding='post')

<class 'list'>
<start>  le règlement de scolarité présente les modalités d' admission à l' école centrale de lyon , les objectifs et les modalités de l' évaluation des connaissances et des compétences de la formation ingénieur , les modalités de diversification de cette formation et les conditions d' obtention des diplômes de l' école centrale de lyon , hors diplômes de master co-accrédités et diplôme d' ingénieur energie en alternance . <stop> [1, 10, 133, 3, 61, 862, 6, 104, 8, 177, 11, 5, 27, 48, 3, 45, 12, 6, 863, 18, 6, 104, 3, 5, 79, 14, 92, 18, 14, 106, 3, 7, 44, 90, 12, 6, 104, 3, 1567, 3, 168, 44, 18, 6, 287, 8, 216, 14, 87, 3, 5, 27, 48, 3, 45, 12, 392, 87, 3, 249, 1191, 18, 24, 8, 90, 1192, 20, 617, 15, 2]


In [11]:
# Fonction qui convertit un mot en son représentant entier
def convert(tokenizer, tensor):
    for t in tensor: # t est un entier élément du tenseur
        if t != 0:
            print ("%d ----> %s" % (t, tokenizer.index_word[t]))
convert(tokenizer, tensor_sentences[-1])

1 ----> <start>
39 ----> comment
7 ----> la
40 ----> mobilité
213 ----> est-elle
1566 ----> vérifiée
17 ----> pour
6 ----> les
99 ----> doubles
87 ----> diplômes
20 ----> en
82 ----> france
4 ----> ?
2 ----> <stop>


# Step 3: Prepare the DataSet

`tokenizer.index_word` est un dictionnaire dont les clés sont des entiers et les valeurs sont des struings (mots du vocabulaire)

In [22]:
print('tensor:')
print(type(tensor_sentences))
print(np.shape(tensor_sentences))
tensor_sentences[0]
print("tokenizer:")
print(type(tokenizer))
print(type(tokenizer.index_word))

vocab_inp_size = len(tokenizer.word_index)
n_data,max_length = tensor_sentences.shape
embedding_dim = 16

print(f"nombre de données: {n_data}\nlongueur max phrases en mots: {max_length}\ntaille du vocabulaire: {vocab_inp_size}\ndimension de l'embedding: {embedding_dim}")

tensor:
<class 'numpy.ndarray'>
(2201, 349)
tokenizer:
<class 'keras_preprocessing.text.Tokenizer'>
<class 'dict'>
nombre de données: 2201
longueur max phrases en mots: 349
taille du vocabulaire: 2557
dimension de l'embedding: 16


# Step 4: Split the train and validation data

In [13]:
from sklearn.model_selection import train_test_split

# Create training and validation sets using an 80/20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(tensor_sentences, tensor_sentences, test_size=0.2)

print(type(input_tensor_train), type(target_tensor_train))
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

# on observe ce qu'il y a dans ces données: si on rééxécute ça change, c'est parce qu'il y a un shuffle aléatoire
convert(tokenizer, input_tensor_train[0])

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
1760 1760 441 441
1 ----> <start>
6 ----> les
83 ----> notes
19 ----> du
398 ----> premier
105 ----> semestre
18 ----> et
262 ----> /
31 ----> ou
10 ----> le
335 ----> placement
20 ----> en
121 ----> première
47 ----> année
21 ----> sont
163 ----> -ils
29 ----> si
496 ----> importants
17 ----> pour
5 ----> l'
177 ----> admission
22 ----> dans
5 ----> l'
120 ----> université
192 ----> partenaire
4 ----> ?
2 ----> <stop>


# Step 5: Create the dataset

Next, call the `tf.data.Dataset` API and create a proper dataset. <br /> Documentation if the `from_tensor_slices`: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices

The given tensors are sliced along their first dimension. This operation preserves the structure of the input tensors, removing the first dimension of each tensor and using it as the dataset dimension. All input tensors must have the same size in their first dimensions.

In [18]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))


print(f"characteristics of dataset")
print(type(dataset), len(dataset))
dataset_iterator_list = list(dataset.as_numpy_iterator())

print("\ncharacteristics of list(dataset.as_numpy_iterator())")

print(type(dataset_iterator_list),len(dataset_iterator_list), "(= steps_per_epoch)")
print(type(dataset_iterator_list[0]), len(dataset_iterator_list[0])) 
print(type(dataset_iterator_list[0][0]), np.shape(dataset_iterator_list[0][0]), "(= BATCH_SIZE,max_length_inp)")

characteristics of dataset
<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'> 1760

characteristics of list(dataset.as_numpy_iterator())
<class 'list'> 1760 (= steps_per_epoch)
<class 'tuple'> 2
<class 'numpy.ndarray'> (349,) (= BATCH_SIZE,max_length_inp)


Validate the shapes of the input and target batches of the newly-created dataset.

In [35]:
# Size of input and target batches
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([512, 349]), TensorShape([512, 349]))

64 (première dimension du tenseur) est le batch tandis que
349 est la longueur maximale d'une phrase (car elles sont toutes paddées)

# Step 6: Encoder Class

In [26]:
# Encoder class
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units


        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)


        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x):
        x = self.embedding(x)
        output, state = self.gru(x)
        return output, state
        # hidden state shape == (batch_size, hidden size)
        # output       shape == (batch_size, max_len, hidden size)

Call the encoder class to check the shapes of the encoder output and hidden state.

In [48]:
encoder = Encoder(vocab_inp_size, embedding_dim, 128)

dummy_inp_enc = tf.random.uniform(
    (6,10),
    minval=0,
    maxval=60,
    dtype=tf.dtypes.int32,
    name="dummy_input_encoder"
)

enc_output, enc_hidden = encoder(dummy_inp_enc)

print('Encoder Output       shape: (batch_size, sequence_length, units) {}'.format(enc_output.shape))
print('Encoder Hidden_state shape: (batch_size, units)                  {}'.format(enc_hidden.shape))

Encoder Output       shape: (batch_size, sequence_length, units) (6, 10, 128)
Encoder Hidden_state shape: (batch_size, units)                  (6, 128)


In [49]:
attention_outputs, attention_scores = tf.keras.layers.Attention()([enc_output, enc_hidden],return_attention_scores=True)
print('Attention output: (batch_size, sequence_length, units)',attention_outputs.shape,'\nAttention scores: (batch_size, sequence_length, units)', attention_scores.shape)
context = attention_outputs * enc_output

Attention output: (batch_size, sequence_length, units) (6, 10, 128) 
Attention scores: (batch_size, sequence_length, units) (6, 10, 6)


In [ ]:
# Decoder class
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units

        self.gru = tf.keras.layers.GRU(self.dec_units,
                                        return_sequences=True,
                                        return_state=True,
                                        recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = tf.keras.layers.Attention()


    def call(self, enc_output):
        attention_outputs, attention_weights = tf.keras.layers.Attention()([enc_output, enc_hidden],return_attention_scores=True)
        context = attention_outputs * enc_output
        

        return 